<a href="https://colab.research.google.com/github/le-incroyable1-dev/tinker_with_Tensorflow/blob/main/IMDB_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84125825/84125825 [==============================] - 2s 0us/step


In [3]:
os.listdir(dataset_dir)

['imdbEr.txt', 'train', 'README', 'test', 'imdb.vocab']

In [4]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['urls_unsup.txt',
 'urls_pos.txt',
 'labeledBow.feat',
 'unsup',
 'pos',
 'urls_neg.txt',
 'neg',
 'unsupBow.feat']

In [5]:
neg_dir = os.path.join(train_dir, 'neg')

In [6]:
sample_file = os.path.join(neg_dir, '24_1.txt')
with open(sample_file) as f:
  print(f.read())

I have to congratulate the genius who approved this one. Edward Furlong, you're not as good as you think mate, you can't grab on every piece of low-cost amateur crap, which sole intention has to be to get some bucks.<br /><br />The filming is bad, and I mean BAD. Anyone with a camera would get the same result, or better.<br /><br />The acting, lets just say: don't go to the supermarket looking for actors. The good ones usually come with a degree or, at least, have some damn experience! The director.. Mr. Jon Keeyes, please find your purpose in life, as a director you simply suck. Your directing is poor, the angles are all messed up (not in a good way), the lines seem as if they're being read out of toilet paper, and the damn music.. it always comes up when it shouldn't and goes out for no apparent reason. And don't go for writer either, by the way. Making movies isn't like serving on a coffeshop, it requires art and skill, things I really doubt you'll ever have.<br /><br />Instead of m

In [7]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [8]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.25, 
    subset='training', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 18750 files for training.


In [9]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.25, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 6250 files for validation.


In [10]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


In [11]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [12]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [13]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)


In [14]:
raw_train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [15]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [16]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />T

In [17]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [18]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
__________________________________________________

In [19]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [20]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
586/586 [==============================] - 12s 20ms/step - loss: 0.6692 - binary_accuracy: 0.6822 - val_loss: 0.6254 - val_binary_accuracy: 0.7653
Epoch 2/10
586/586 [==============================] - 11s 19ms/step - loss: 0.5622 - binary_accuracy: 0.7947 - val_loss: 0.5128 - val_binary_accuracy: 0.8168
Epoch 3/10
586/586 [==============================] - 11s 18ms/step - loss: 0.4585 - binary_accuracy: 0.8401 - val_loss: 0.4342 - val_binary_accuracy: 0.8410
Epoch 4/10
586/586 [==============================] - 11s 19ms/step - loss: 0.3894 - binary_accuracy: 0.8634 - val_loss: 0.3868 - val_binary_accuracy: 0.8514
Epoch 5/10
586/586 [==============================] - 11s 19ms/step - loss: 0.3445 - binary_accuracy: 0.8759 - val_loss: 0.3562 - val_binary_accuracy: 0.8606
Epoch 6/10
586/586 [==============================] - 11s 19ms/step - loss: 0.3128 - binary_accuracy: 0.8866 - val_loss: 0.3361 - val_binary_accuracy: 0.8672
Epoch 7/10
586/586 [==============================] 

In [21]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 12s 15ms/step - loss: 0.3135 - binary_accuracy: 0.8724
Loss:  0.3134799897670746
Accuracy:  0.8723599910736084


In [22]:
#export the model

export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

782/782 [==============================] - 9s 12ms/step - loss: 0.3135 - accuracy: 0.8724
0.8723599910736084


In [25]:
examples = [
  "The movie was amazing!",
  "The movie was okay.",
  "The movie was horrible...",
  "I did not like the movie",
  "I loved the movie so much!"
]

export_model.predict(examples)

1/1 [==============================] - 0s 37ms/step


array([[0.61799955],
       [0.43500537],
       [0.36306998],
       [0.45027247],
       [0.5866078 ]], dtype=float32)